[![View in Deepnote](https://deepnote.com/static/buttons/view-in-deepnote-white.svg)](https://deepnote.com/viewer/github/katetruman/MultiAgentEC/blob/master/ExpectationExamples/Individual%20Expectations.ipynb)

# Individual expectations

This notebook explores expectations related to our organ transplant scenario. If you have not yet done so, please refer to **AgentsAndEvents.ipynb** for important events and fluents used in this scenario, and **Expectations.ipynb** for a discussion of expectations in this context. Both of these notebooks are located in the parent repository, **MultipleAgentsEC**.

### Set up

We need to set up our environment by loading in our event and fluent declarations from **dec:notation.pl** and **AE.pl**. The latter contains the code from **AgentsAndEvents.ipynb** the last time that notebook was run.

```diff
- IMPORTANT NOTE: you should run AgentsAndEvents.ipynb prior to running this notebook to ensure that AE.pl contains up to date predicates! You must run the notebook, not just update and save it.
```

In [1]:
?- cd('~/work'), ['dec:notation'].
?- initialiseDEC.
?- retractall(happensAtNarrative(_,_)).
?- ['AE'].

true.
false.
true.
true.

Individual hospital agents may have expectations specific to a given scenario that do not fit into one of our other expectation categories. Examples of some organ transplant specific expectations are shown below.

Otago hospital expects that once a request to add a patient to the organ waiting list has been accepted, they will not have to wait more than 25 time periods for a transplant:

In [2]:
% File: maxWaitingTime.pl
initially("Otago":exp_rule(happ(receive(monSys, waitAccept(ID))), within(and([happ(receive(Location, transplantOutcome(MatchID, _))), 
donorFound(MatchID, ID, _, Location, _, _)]), 25), dependent, 
"Individual":"Patients will not have to wait more than 25 time periods for a transplant")).

Wellington hospital expects that waiting list add requests will not involve more than two organs per patient:

In [3]:
% File: maximumOrgans.pl
initially("Wellington":exp_rule(happ(send(monSys, waitAddReq(_, Organs, _))), and([condition(length(Organs, Len)), condition(Len =< 2)]), dependent, 
"Individual":"Patients will not be added to the waiting list for more than 2 organs at once")).

Auckland hospital expects that donor Offers will not be made for patients with cancer:

In [4]:
% File: noCancerOffers.pl
initially("Auckland":exp_rule(happ(send(monSys, donorOffer(PatID, Organs, Details))), not(and([condition(member(condition:D, Details)), 
condition(member("Cancer", D))])), dependent, "Individual":"Donor offers will not be made for patients diagnosed with cancer")).

Our test narrative is as follows:
- At time period 1, Otago sends a waiting list add request for patient 101 to **monSys**.
- At time period 3, Otago sends a waiting list add request for patient  to **monSys**.
- At time period 5, Wellington sends a waiting list add request for 2 organs for a patient to **monSys**.
- At time period 7, Wellington sends a waiting list add request for 3 organs for a patient to **monSys**.
- At time period 9, Auckland sends a donor offer for a patient with cancer to **monSys**.
- At time period 11, Auckland sends a donor offer for patient 202 to **monSys**.
- At time period 13, **monSys** matches the donor offer from patient 202 from Auckland to waiting patient 102 from Otago.

In [5]:
% File: narrative.pl
happensAtNarrative("Otago":send(monSys, waitAddReq(101, ["kidney", "liver"], [bloodType:"O"])),1).
happensAtNarrative("Otago":send(monSys, waitAddReq(102, ["kidney", "heart", "liver"], [bloodType:"A"])),3).
happensAtNarrative("Wellington":send(monSys, waitAddReq(103, ["heart", "pancreas"], [bloodType:"B"])),5).
happensAtNarrative("Wellington":send(monSys, waitAddReq(104, ["heart", "pancreas", "lungs"], [bloodType:"AB"])),7).
happensAtNarrative("Auckland":send(monSys, donorOffer(201, ["heart"], [bloodType:"A", condition:["HIV", "Cancer"]])), 9).
happensAtNarrative("Auckland":send(monSys, donorOffer(202, ["kidney", "liver", "pancreas"], [bloodType:"O"])),11).
happensAtNarrative(monSys:match(701, "Auckland", 202, "Otago", 102, "Wellington", ["kidney", "liver"], []),13).
happensAtNarrative("Auckland":send(monSys, acceptMatch(701)), 15).
happensAtNarrative("Wellington":send(monSys, acceptMatch(701)), 15).
happensAtNarrative("Otago":send(monSys, acceptMatch(701)), 15).
happensAtNarrative("Wellington":transplant(701, success), 20).

In [6]:
?- run(30).

true.

At time period 4, Otago is notified that their waiting list add request for patient 101 has been accepted. This creates the expectation that a donor match will be found for patient 101 within 25 time periods. Likewise, at time period 6 Otago is notified that their add request for patient 102 has been accepted, which creates another expectation.

In [7]:
?- T = 4, happensAt("Otago", Event, T).
?- T = 4, holdsAt("Otago", exp(_,_,_,Condition,_,Message), T).
?- T = 6, happensAt("Otago", Event, T).
?- T = 6, holdsAt("Otago", exp(_,_,_,Condition,_,Message), T).

T = 4, Event = receive(monSys, waitAccept(101)) .
T = 4, Condition = within(and([Functor(14504077,1,receive(_1758, transplantOutcome(_1764, _1766))), Functor(14639885,6,_1764,101,_1780,_1758,_1784,_1786)]), 25), Message = :(b'Individual', b'Patients will not have to wait more than 25 time periods for a transplant') .
T = 6, Event = receive(monSys, waitAccept(102)) .
T = 6, Condition = within(and([Functor(14504077,1,receive(_1758, transplantOutcome(_1764, _1766))), Functor(14639885,6,_1764,101,_1780,_1758,_1784,_1786)]), 23), Message = :(b'Individual', b'Patients will not have to wait more than 25 time periods for a transplant') ;
T = 6, Condition = within(and([Functor(14504077,1,receive(_1758, transplantOutcome(_1764, _1766))), Functor(14639885,6,_1764,102,_1780,_1758,_1784,_1786)]), 25), Message = :(b'Individual', b'Patients will not have to wait more than 25 time periods for a transplant') .

At time period 5, Wellington sends a waiting list add request for a patient needing two organs. This fulfils the expectation that a waiting list add request will not be for more than 2 organs. At time period 7, Wellington sends a waiting list add request for a patient needing three organs, which violates the expectation.

In [8]:
?- T = 5, happensAt("Wellington", Event, T), Event = send(_, _).
?- T = 5, happensAt("Wellington", fulf(_ ,_, _, FulfCondition, _, Message), T).
?- T = 7, happensAt("Wellington", Event, T), Event = send(_,_).
?- T = 7, happensAt("Wellington", viol(_, _, _, ViolCondition, _, Message), T).

T = 5, Event = send(monSys, waitAddReq(103, [b'heart', b'pancreas'], [Functor(188685,2,bloodType,b'B')])) .
T = 5, FulfCondition = and([Functor(14479501,1,length([b'heart', b'pancreas'], 2)), Functor(14479501,1,=<(2, 2))]), Message = :(b'Individual', b'Patients will not be added to the waiting list for more than 2 organs at once') .
T = 7, Event = send(monSys, waitAddReq(104, [b'heart', b'pancreas', b'lungs'], [Functor(188685,2,bloodType,b'AB')])) .
T = 7, ViolCondition = and([Functor(14479501,1,length([b'heart', b'pancreas', b'lungs'], 3)), Functor(14479501,1,=<(3, 2))]), Message = :(b'Individual', b'Patients will not be added to the waiting list for more than 2 organs at once') .

When Auckland sends a donor offer for a patient with cancer at time period 9, it violates the relevant expectation. When Auckland sends a donor offer for a different patient who is not noted to have cancer at time period 11, the relevant expectation is fulfilled.

In [9]:
?- T = 9, happensAt("Auckland", Event, T), Event = send(_, _).
?- T = 9, happensAt("Auckland", viol(_, _, _, ViolCondition, _, Message), T).
?- T = 11, happensAt("Auckland", Event, T), Event = send(_, _).
?- T = 11, happensAt("Auckland", fulf(_, _, _, FulfCondition, _, Message), T).

T = 9, Event = send(monSys, donorOffer(201, [b'heart'], [Functor(188685,2,bloodType,b'A'), Functor(188685,2,condition,[b'HIV', b'Cancer'])])) .
T = 9, ViolCondition = not(and([Functor(14479501,1,member(:(condition, [b'HIV', b'Cancer']), [Functor(188685,2,bloodType,b'A'), Functor(188685,2,condition,[b'HIV', b'Cancer'])])), Functor(14479501,1,member(b'Cancer', [b'HIV', b'Cancer']))])), Message = :(b'Individual', b'Donor offers will not be made for patients diagnosed with cancer') .
T = 11, Event = send(monSys, donorOffer(202, [b'kidney', b'liver', b'pancreas'], [Functor(188685,2,bloodType,b'O')])) .
T = 11, FulfCondition = not(and([Functor(14479501,1,member(:(condition, [b'Cancer']), [Functor(188685,2,bloodType,b'O')])), Functor(14479501,1,member(b'Cancer', [b'Cancer']))])), Message = :(b'Individual', b'Donor offers will not be made for patients diagnosed with cancer') .

At time period 13, **monSys** matches the donor offer for patient 202 from Auckland to the waiting patient 102 from Otago, and specifies the transplant location as Wellington. This causes messages about the match to be sent to each of the three hospitals, which all send acceptance messages at time period 15. The transplant successfully takes place at time period 20, and at time period 21 Wellington sends a message to **monSys** and Otago and Auckland hospitals, notifying them of the transplant.

In [10]:
?- T = 20, happensAt("Wellington", Event, T).
?- T = 21, happensAt("Wellington", Event, T).

T = 20, Event = transplant(701, success) .
T = 21, Event = send(b'Auckland', transplantOutcome(701, success)) ;
T = 21, Event = send(b'Otago', transplantOutcome(701, success)) ;
T = 21, Event = send(monSys, transplantOutcome(701, success)) .

At time period 22, Otago receives the `transplantOutcome` notification, which fulfils its expectation that patient 102 will not have to wait more than 25 periods for an organ donation.

In [11]:
?- T = 22, happensAt("Otago", Event, T).

T = 22, Event = fulf(happ(receive(monSys, waitAccept(102))), within(and([Functor(14504077,1,receive(b'Wellington', transplantOutcome(701, success))), Functor(14639885,6,701,102,b'Auckland',b'Wellington',[b'kidney', b'liver'],[])]), 25), 6, within(and([Functor(14504077,1,receive(b'Wellington', transplantOutcome(701, success))), Functor(14639885,6,701,102,b'Auckland',b'Wellington',[b'kidney', b'liver'],[])]), 9), dependent, :(b'Individual', b'Patients will not have to wait more than 25 time periods for a transplant')) ;
T = 22, Event = receive(b'Wellington', transplantOutcome(701, success)) .

At time period 29, Otago's expectation that patient 101 will not have to wait more than 25 time periods from the wait list acceptance message for an organ donation is violated.

In [12]:
?- happensAt("Otago", viol(_, _, _, ViolCondition, _, Message), 29).

ViolCondition = within(and([Functor(14504077,1,receive(_1700, transplantOutcome(_1706, _1708))), Functor(14639885,6,_1706,101,_1722,_1700,_1726,_1728)]), 0), Message = :(b'Individual', b'Patients will not have to wait more than 25 time periods for a transplant') .

Note that the different agents were able to have different expectation rules which could be successfully trigger, violated or fulfilled by messages sent between the agents.

# Visualisation

In [13]:
% File: helperVisualisation.pl
initialAgents(AgentList):- findall(Agent, initially(monSys:agent(Agent)), AgentList).
allAgents(AgentList):- initialAgents(InitialList), append([monSys], InitialList, AgentList).

visualisationLabel(waitAddReq, "Waiting list add request", ["patient ID", "organs", "details"]).
visualisationLabel(waitAccept, "waiting list acceptance", ["patient ID"]).
visualisationLabel(donorOffer, "Donation offer", ["patient ID", "organs", "details"]).
visualisationLabel(acceptMatch, "Match acceptance", ["match ID"]).
visualisationLabel(confirmedNotification, "Match confirmation", ["match ID"]).
visualisationLabel(recipientFound, "Recipient found notification", ["match ID", "recipient ID", "donor hospital", "transplant location", 
"organs", "details"]).
visualisationLabel(donorFound, "Donor found notification", ["match ID", "donor ID", "recipient hospital", "transplant location", 
"organs", "details"]).
visualisationLabel(locationSelected, "Transplant location selected", ["match ID", "donor hospital", "recipient hospital",
"organs", "details"])
visualisationLabel(transplantOutcome, "Transplant outcome", ["match ID", "outcome"]).
visualisationLabel(transplanted, "Transplant", ["match ID", "outcome"]).
visualisationLabel(waiting, "Waiting", ["patient ID", "hospital", "organs", "details", "start time"]).

In [16]:
% Output: VisualisationSupport.txt
?- allAgents(Agents).
?- visualisationLabel(FluentName, FluentMessage, Arguments).

Agents = [ monSys, b'Otago', b'Christchurch', b'Wellington', b'Auckland' ] .
FluentName = waitAddReq, FluentMessage = b'Waiting list add request', Arguments = [ b'patient ID', b'organs', b'details' ] ;
FluentName = waitAccept, FluentMessage = b'waiting list acceptance', Arguments = [ b'patient ID' ] ;
FluentName = donorOffer, FluentMessage = b'Donation offer', Arguments = [ b'patient ID', b'organs', b'details' ] ;
FluentName = acceptMatch, FluentMessage = b'Match acceptance', Arguments = [ b'match ID' ] ;
FluentName = confirmedNotification, FluentMessage = b'Match confirmation', Arguments = [ b'match ID' ] ;
FluentName = recipientFound, FluentMessage = b'Recipient found notification', Arguments = [ b'match ID', b'recipient ID', b'donor hospital', b'transplant location', b'organs', b'details' ] ;
FluentName = donorFound, FluentMessage = b'Donor found notification', Arguments = [ b'match ID', b'donor ID', b'recipient hospital', b'transplant location', b'organs', b'details' ] ;
FluentName

In [17]:
% Output: VisualisationEvents.txt

?- happensAt(SendAgent, Event, T), Event = send(_, _).
?- happensAt(ReceiveAgent, Event, T), Event = receive(_, _).
?- happensAt(ViolAgent, Event, T), Event = viol(_,_,_,_,_,_).
?- happensAt(FulfAgent, Event, T), Event = fulf(_,_,_,_,_,_).
?- happensAt(OtherAgent, Event, T), Event \= send(_, _), Event \= receive(_, _), Event \= viol(_, _, _, _, _, _), Event \= fulf(_, _, _, _, _, _).

SendAgent = b'Otago', Event = send(monSys, waitAddReq(101, [b'kidney', b'liver'], [Functor(188685,2,bloodType,b'O')])), T = 1 ;
SendAgent = b'Otago', Event = send(monSys, waitAddReq(102, [b'kidney', b'heart', b'liver'], [Functor(188685,2,bloodType,b'A')])), T = 3 ;
SendAgent = b'Wellington', Event = send(monSys, waitAddReq(103, [b'heart', b'pancreas'], [Functor(188685,2,bloodType,b'B')])), T = 5 ;
SendAgent = b'Wellington', Event = send(monSys, waitAddReq(104, [b'heart', b'pancreas', b'lungs'], [Functor(188685,2,bloodType,b'AB')])), T = 7 ;
SendAgent = b'Auckland', Event = send(monSys, donorOffer(201, [b'heart'], [Functor(188685,2,bloodType,b'A'), Functor(188685,2,condition,[b'HIV', b'Cancer'])])), T = 9 ;
SendAgent = b'Auckland', Event = send(monSys, donorOffer(202, [b'kidney', b'liver', b'pancreas'], [Functor(188685,2,bloodType,b'O')])), T = 11 ;
SendAgent = b'Auckland', Event = send(monSys, acceptMatch(701)), T = 15 ;
SendAgent = b'Wellington', Event = send(monSys, acceptMatch(701))

In [15]:
% Output: 
?- holdsAt(FluentAgent, Fluent, T), Fluent = waiting(_,_,_,_,_), 0 =< T, T =< 9.
?- holdsAt(FluentAgent, Fluent, T), Fluent = waiting(_,_,_,_,_), 10 =< T, T =< 19.
?- holdsAt(FluentAgent, Fluent, T), Fluent = waiting(_,_,_,_,_), 20 =< T, T =< 29.

FluentAgent = monSys, Fluent = waiting(101, b'Otago', [b'kidney', b'liver'], [Functor(188685,2,bloodType,b'O')], 2), T = 3 ;
FluentAgent = monSys, Fluent = waiting(101, b'Otago', [b'kidney', b'liver'], [Functor(188685,2,bloodType,b'O')], 2), T = 4 ;
FluentAgent = monSys, Fluent = waiting(101, b'Otago', [b'kidney', b'liver'], [Functor(188685,2,bloodType,b'O')], 2), T = 5 ;
FluentAgent = monSys, Fluent = waiting(102, b'Otago', [b'kidney', b'heart', b'liver'], [Functor(188685,2,bloodType,b'A')], 4), T = 5 ;
FluentAgent = monSys, Fluent = waiting(101, b'Otago', [b'kidney', b'liver'], [Functor(188685,2,bloodType,b'O')], 2), T = 6 ;
FluentAgent = monSys, Fluent = waiting(102, b'Otago', [b'kidney', b'heart', b'liver'], [Functor(188685,2,bloodType,b'A')], 4), T = 6 ;
FluentAgent = monSys, Fluent = waiting(101, b'Otago', [b'kidney', b'liver'], [Functor(188685,2,bloodType,b'O')], 2), T = 7 ;
FluentAgent = monSys, Fluent = waiting(102, b'Otago', [b'kidney', b'heart', b'liver'], [Functor(188685,2,

In [19]:
% PYTHON
%f = open("/work/output_files/VisualisationEvents.txt", "r")
%text = f.read()
%print(text)
%f.close()

ERROR: Script gave error invalid syntax (<string>, line 1)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=32f94018-a4da-40ef-8c9f-8983d73811c8' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>